In [93]:
import pandas as pd
import datetime
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from collections import Counter

In [122]:
model_date_start=pd.to_datetime("2023-06-21 00:00:00")  #  + pd.Timedelta(hours=5)
model_date_end=pd.to_datetime("2023-07-25 23:50:00")
num_last_days_for_testing = 2

In [123]:
df = pd.read_excel(
    "output/resampled_df_10_min.xlsx", index_col=[0]
)

In [124]:
df = df[df['time'].between(model_date_start, model_date_end)]

In [125]:
df["weekday"] = df["time"].dt.dayofweek
df["hour"] = df["time"].dt.hour
df["day"] = df["time"].dt.day

In [126]:
df

,time,location,weekday,hour,day
6768,2023-06-21 00:00:00,"Lokerenpassage, 18",2,0,21
6769,2023-06-21 00:10:00,"Lokerenpassage, 18",2,0,21
6770,2023-06-21 00:20:00,"Lokerenpassage, 18",2,0,21
6771,2023-06-21 00:30:00,"Lokerenpassage, 18",2,0,21
6772,2023-06-21 00:40:00,"Lokerenpassage, 18",2,0,21
...,...,...,...,...,...
11803,2023-07-25 23:10:00,"Lokerenpassage, 18",1,23,25
11804,2023-07-25 23:20:00,"Lokerenpassage, 18",1,23,25
11805,2023-07-25 23:30:00,"Lokerenpassage, 18",1,23,25
11806,2023-07-25 23:40:00,"Lokerenpassage, 18",1,23,25


In [127]:
le = preprocessing.LabelEncoder()
df.location = le.fit_transform(df.location)

In [128]:
train_start_date = model_date_start
train_end_date = model_date_end - pd.Timedelta(days=days_test)
test_start_date = model_date_end - pd.Timedelta(days=days_test) + pd.Timedelta(minutes=10)
test_end_date = model_date_end

# Create masks to filter the data based on dates
train_mask = (df['time'].between(train_start_date, train_end_date))
test_mask = (df['time'].between(test_start_date, test_end_date))

# # Split the data into train and test sets
X_train = df.loc[train_mask, ["weekday", "hour", "day"]]
y_train = df.loc[train_mask, "location"]
X_test = df.loc[test_mask, ["weekday", "hour", "day"]]
y_test = df.loc[test_mask, "location"]

In [129]:
X_train

,weekday,hour,day
6768,2,0,21
6769,2,0,21
6770,2,0,21
6771,2,0,21
6772,2,0,21
...,...,...,...
11515,6,23,23
11516,6,23,23
11517,6,23,23
11518,6,23,23


In [130]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

In [131]:
model_accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (model_accuracy * 100.0))


Accuracy: 92.36%


In [132]:
import warnings
warnings.filterwarnings('always')  # "error", "ignore", "always", "default", "module" or "once"

class_report = classification_report(y_test, predictions)
print(f"Classification report: \n{class_report}")

Classification report: 
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         0
           5       0.96      0.96      0.96       278
           7       0.00      0.00      0.00         7
           9       0.00      0.00      0.00         3

    accuracy                           0.92       288
   macro avg       0.24      0.24      0.24       288
weighted avg       0.93      0.92      0.93       288



C:\Users\Sander\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Sander\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Sander\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Sander\anaconda3\lib\site-packages\sklearn\metrics\

In [40]:
import pandas as pd

# Option 1:
data = {
    "timestamp": "2013-10-31T12:13:16.892Z"
  },{
    "timestamp": "2013-10-31T12:14:16.909Z"
  },{
    "timestamp": "2013-10-31T12:15:17.018Z"
  },{
    "timestamp": "2013-10-31T12:16:17Z"
  },{
    "timestamp": "2013-10-31T12:17:16.937Z"
  },{
    "timestamp": "2013-10-31T12:18:17.036Z"
  },{
    "timestamp": "2013-10-31T12:19:18.837Z"
}

df = pd.DataFrame(
    data, columns=["timestamp"]
)

df["timestamp"] = pd.to_datetime(
    df["timestamp"], format="%Y-%m-%dT%H:%M:%S.%f"
)

df.head()

,timestamp
0,2013-10-31 12:13:16.892000+00:00
1,2013-10-31 12:14:16.909000+00:00
2,2013-10-31 12:15:17.018000+00:00
3,2013-10-31 12:16:17+00:00
4,2013-10-31 12:17:16.937000+00:00


In [41]:
import pandas as pd

# Option 2:
data = {
    "timestamp": "2013-10-31T12:13:16.892Z"
  },{
    "timestamp": "2013-10-31T12:14:16.909Z"
  },{
    "timestamp": "2013-10-31T12:15:17.018Z"
  },{
    "timestamp": "2013-10-31T12:16:17Z"
  },{
    "timestamp": "2013-10-31T12:17:16.937Z"
  },{
    "timestamp": "2013-10-31T12:18:17.036Z"
  },{
    "timestamp": "2013-10-31T12:19:18.837Z"
}

df = pd.DataFrame(
    data, columns=["timestamp"]
)

df["timestamp"] = pd.to_datetime(
    df["timestamp"], format="%Y-%m-%dT%H:%M:%S"
)

df.head()

,timestamp
0,2013-10-31 12:13:16.892000+00:00
1,2013-10-31 12:14:16.909000+00:00
2,2013-10-31 12:15:17.018000+00:00
3,2013-10-31 12:16:17+00:00
4,2013-10-31 12:17:16.937000+00:00


In [43]:
import pandas as pd

# Option 3:
data = {
    "timestamp": "2013-10-31T12:13:16.892Z"
  },{
    "timestamp": "2013-10-31T12:14:16.909Z"
  },{
    "timestamp": "2013-10-31T12:15:17.018Z"
  },{
    "timestamp": "2013-10-31T12:16:17Z"
  },{
    "timestamp": "2013-10-31T12:17:16.937Z"
  },{
    "timestamp": "2013-10-31T12:18:17.036Z"
  },{
    "timestamp": "2013-10-31T12:19:18.837Z"
}

df = pd.DataFrame(
    data, columns=["timestamp"]
)

df["timestamp"] = pd.to_datetime(
    df["timestamp"], format="%Y-%m-%dT%H:%M:%S.%fZ"
)

df.head()

ValueError: time data '2013-10-31T12:16:17Z' does not match format '%Y-%m-%dT%H:%M:%S.%fZ' (match)

In [44]:
import pandas as pd

# Option 4:
data = {
    "timestamp": "2013-10-31T12:13:16.892Z"
  },{
    "timestamp": "2013-10-31T12:14:16.909Z"
  },{
    "timestamp": "2013-10-31T12:15:17.018Z"
  },{
    "timestamp": "2013-10-31T12:16:17Z"
  },{
    "timestamp": "2013-10-31T12:17:16.937Z"
  },{
    "timestamp": "2013-10-31T12:18:17.036Z"
  },{
    "timestamp": "2013-10-31T12:19:18.837Z"
}

df = pd.DataFrame(
    data, columns=["timestamp"]
)

df["timestamp"] = pd.to_datetime(
    df["timestamp"], format="%Y-%m-%dT%H:%M:%SZ"
)

df.head()

ValueError: time data '2013-10-31T12:13:16.892Z' does not match format '%Y-%m-%dT%H:%M:%SZ' (match)